<a href="https://colab.research.google.com/github/elnaggarx/RAG-model-from-MKDocs/blob/main/MKDocs_RAG_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain longchain_classic langchain-google-genai langchain-community langchain-text-splitters chromadb python-dotenv

ERROR: Could not find a version that satisfies the requirement longchain_classic (from versions: none)
ERROR: No matching distribution found for longchain_classic


In [ ]:
import os
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter , RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from dotenv import load_dotenv
import shutil
import openai
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
load_dotenv()

False

In [ ]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Ingestion Pipeline

In [ ]:
path = "/content/drive/MyDrive/docs"

#Markdown text loader function

def load_documents(docs_path="/content/drive/MyDrive/docs"):
    """Load all text files from the docs directory"""
    print(f"Loading documents from {docs_path}...")


    if not os.path.exists(docs_path):
        raise FileNotFoundError(f"The directory {docs_path} does not exist. Please create it and add your company files.")


    loader = DirectoryLoader(
        path=docs_path,
        glob="**/*.md",  # Keep recursive glob pattern
        loader_cls=TextLoader
        # Removed recursive=True, as glob="**/*.md" should handle recursion
    )

    documents = loader.load()

    # Add debug print for the number of documents found
    print(f"Total documents found by loader: {len(documents)}")
    for i, doc in enumerate(documents):
        if i < 5: # Limit to first 5 for brevity
            print(f"  Document {i+1} source: {doc.metadata['source']}")
        elif i == 5:
            print("  ... (more documents)")


    if len(documents) == 0:
        raise FileNotFoundError(f"No .md files found in {docs_path} or its subdirectories. Please add your company documents.")


    for i, doc in enumerate(documents):  # Show first 2 documents
        print(f"\nDocument {i+1}:")
        print(f"  Source: {doc.metadata['source']}")
        print(f"  Content length: {len(doc.page_content)} characters")
        print(f"  Content preview: {doc.page_content[:100]}...")
        print(f"  metadata: {doc.metadata}")

    return documents
documents = load_documents(path)

Loading documents from /content/drive/MyDrive/docs...
Total documents found by loader: 19
  Document 1 source: /content/drive/MyDrive/docs/index.md
  Document 2 source: /content/drive/MyDrive/docs/getting-started.md
  Document 3 source: /content/drive/MyDrive/docs/user-guide/cli.md
  Document 4 source: /content/drive/MyDrive/docs/user-guide/installation.md
  Document 5 source: /content/drive/MyDrive/docs/user-guide/configuration.md
  ... (more documents)

Document 1:
  Source: /content/drive/MyDrive/docs/index.md
  Content length: 3281 characters
  Content preview: # MkDocs

Project documentation with&nbsp;Markdown.

---

MkDocs is a **fast**, **simple** and **dow...
  metadata: {'source': '/content/drive/MyDrive/docs/index.md'}

Document 2:
  Source: /content/drive/MyDrive/docs/getting-started.md
  Content length: 6529 characters
  Content preview: # Getting Started with MkDocs

An introductory tutorial!

---

## Installation

To install MkDocs, r...
  metadata: {'source': '/content/d

In [ ]:
#Text splitter function

"""
# Why Recursive is the best option of splitters for this project ?

Because MkDocs documentation has structured Markdown, and recursive splitting preserves paragraphs, lists, and explanations without breaking meaning.
This produces clean semantic chunks, which dramatically improves search accuracy and answer quality in your RAG system.
"""
def split_text(documents):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500 , length_function = len , add_start_index = True)
  texts = text_splitter.split_documents(documents)
  print(f"Total chunks after split: {len(texts)}")
  print(texts[:30])
  return texts
chunks = split_text(documents)



Total chunks after split: 508
[Document(metadata={'source': '/content/drive/MyDrive/docs/index.md', 'start_index': 0}, page_content='# MkDocs\n\nProject documentation with&nbsp;Markdown.\n\n---\n\nMkDocs is a **fast**, **simple** and **downright gorgeous** static site\ngenerator that\'s geared towards building project documentation. Documentation\nsource files are written in Markdown, and configured with a single YAML\nconfiguration file. Start by reading the [introductory tutorial], then check the\n[User Guide] for more information.\n\n[introductory tutorial]: getting-started.md\n[User Guide]: user-guide/README.md\n\n<div class="text-center">\n<a href="getting-started/" class="btn btn-primary" role="button">Getting Started</a>\n<a href="user-guide/" class="btn btn-primary" role="button">User Guide</a>\n</div>\n\n<div class="pt-2 pb-4 px-4 my-4 bg-body-tertiary rounded-3">\n<h2 class="display-4 text-center">Features</h2>'), Document(metadata={'source': '/content/drive/MyDrive/docs/inde

In [ ]:
# Load the API key from Colab's secrets manager
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
CHROMA_PATH = "/tmp/chroma_db" # Changed the path to a temporary directory

def save_to_chroma(chunks):
  print("Saving to ChromaDB with Gemini embeddings...")
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Explicitly pass the API key
  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.environ["GOOGLE_API_KEY"])
  db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=CHROMA_PATH)
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH} using Gemini embeddings.")
save_to_chroma(chunks)

Saving to ChromaDB with Gemini embeddings...


GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerProj

# Query Pipeline

In [ ]:
def load_chroma():
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",
        google_api_key=os.environ["GOOGLE_API_KEY"]
    )
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )
    return db

db = load_chroma()

/tmp/ipython-input-2130980661.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(


In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 5})


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [ ]:
query = "How do I create a new MkDocs project?"
response = qa_chain.run(query)
print(response)

/tmp/ipython-input-2656072333.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = qa_chain.run(query)


GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerProject